In [36]:
import re
import os
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import sys

In [38]:
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)
sys.path.append(project_dir)

In [2]:
df = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\01_raw\tweets_details2023-03-15_20-43-36.csv")

In [41]:
# clean text 
from data_processing.preprocess_tweets import TextCleaner

cleaner = TextCleaner()
df['cleaned_text'] = df['text'].apply(cleaner.clean_text)


In [42]:
df.head(5)

tweet_id                   creation_date  \
0  7.485231e+17  Thu Jun 30 14:26:32 +0000 2016   
1  7.484731e+17  Thu Jun 30 11:07:57 +0000 2016   
2  7.485466e+17  Thu Jun 30 15:59:48 +0000 2016   
3  7.485004e+17  Thu Jun 30 12:56:24 +0000 2016   
4  7.485809e+17  Thu Jun 30 18:16:08 +0000 2016   

                                                text  \
0  #royalmail offers Compulsory new “Chargeable” ...   
1  Great #friends realize your #love of all thing...   
2  My #dilevery of #easyskinz with #royalmail #ip...   
3  #royalmail ridiculous rule not being able to s...   
4  📮 #RoyalMail MarketReach Report Dispels #Direc...   

                                           media_url video_url language  \
0                                                NaN       NaN       en   
1                                                NaN       NaN       en   
2                                                NaN       NaN       en   
3                                                NaN       NaN       en   
4  ['https://pbs.twimg.com/media/CmL0Z4qWIAAJDDe....       NaN       en   

   favorite_count  retweet_count  reply_count  quote_count  ...  \
0             0.0            0.0          0.0          0.0  ...   
1             1.0            0.0          0.0          0.0  ...   
2             1.0            0.0          0.0          0.0  ...   
3             0.0            0.0          0.0          0.0  ...   
4             1.0            0.0          0.0          0.0  ...   

                                    user.description  \
0  Selro provides all the tools to start and grow...   
1                                                NaN   
2                                instagram: thimothy   
3  Consumer rights activist— holding companies to...   
4  #UK based #printing company.         https://t...   

                         user.external_url  user.number_of_tweets  user.bot  \
0                     http://www.selro.com                 1036.0     False   
1  http://our-labour-of-love.blogspot.com/                 1973.0     False   
2                                      NaN                 1117.0     False   
3                                      NaN                16688.0     False   
4                 http://www.printmr.co.uk                 9769.0     False   

   user.timestamp  user.has_nft_avatar user detail  \
0    1.239193e+09                False  NaN    NaN   
1    1.368150e+09                False  NaN    NaN   
2    1.286693e+09                False  NaN    NaN   
3    1.398434e+09                False  NaN    NaN   
4    1.305716e+09                False  NaN    NaN   

                                        cleaned_text  \
0  offer Compulsory new Chargeable IntegratorCont...   
1  Great friend realize love thing written Love g...   
2  dilevery easyskinz iphone6 mehappy blackmamba ...   
3  ridiculous rule able send perfume etc post eve...   
4  MarketReach Report Dispels DirectMail Engageme...   

                                      topic_keywords  
0  stamp, philately, postage, postagestamps, stam...  
1  postbox, letter, post, box, letterbox, santa, ...  
2  post, extra, strikes, postalstrike, shopping, ...  
3  parcel, help, item, delivery, lost, day, deliv...  
4  share, price, low, profit, company, executive,...  

[5 rows x 41 columns]

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Create a CountVectorizer object to convert the cleaned text into a bag-of-words representation
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cleaned_text'])

# Train the LDA model
lda_model = LatentDirichletAllocation(n_components=100, random_state=42)
lda_model.fit(X)

# Get the feature names from the vectorizer
feature_names = vectorizer.get_feature_names_out()

topic_keywords = []

# Print the top 10 words for each topic
for topic_idx, topic in enumerate(lda_model.components_):
    print("Topic %d:" % (topic_idx))
    top_features = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    print(", ".join(top_features))
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    topic_keywords.append(', '.join(top_words))

# add topic keywords to dataframe
df['topic_keywords'] = [topic_keywords[i] for i in lda_model.transform(vectorizer.transform(df['cleaned_text'])).argmax(axis=1)]



Topic 0:
united, kingdom, vehicle, child, amp, career, flag, building, dc, london
Topic 1:
drone, costoflivingcrisis, nurse, job, railstrikes, btstrike, recession, indyref2, barristerstrike, busdriverstrike
Topic 2:
change, cyber, hell, incident, tariff, review, know, mail, respect, mailmark
Topic 3:
absolute, joke, mail, want, disgrace, dont, redirection, bbcnews, get, skynews
Topic 4:
nhs, public, privatisation, keyworkers, government, tories, amp, labour, tory, water
Topic 5:
town, post, cash, finished, 500, year, christmasiscoming, office, night, oxford
Topic 6:
help, phone, call, still, waiting, please, complaint, get, someone, minute
Topic 7:
dad, boy, kindness, heaven, today, nationalise, christmas2020, via, fatherchristmas, kensingtonroyal
Topic 8:
post, advice, amp, get, day, snow, home, parcel, getting, mail
Topic 9:
delivery, issue, day, ceo, simon, next, home, delivered, week, new
Topic 10:
record, marketing, amp, mail, people, letter, shall, doubt, directmail, post
Topic 1

In [28]:
df.to_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\02_intermediate\tweet_topics.csv")